In [4]:
import sys
import os

# Získáme cestu k aktuální složce
current_dir = os.getcwd()

# Získáme cestu k NADŘAZENÉ složce (o úroveň výš)
parent_dir = os.path.dirname(current_dir)

# Přidáme nadřazenou složku do cest, kde Python hledá
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Teď už by měl Python "vidět" složku pzs_lib jako balíček
import pzs_lib as pzs
print("PZS Lib načtena úspěšně!")

import numpy as np
import matplotlib.pyplot as plt
import wfdb  # pip install wfdb
import pzs_lib as pzs  # Naše knihovna

# Aby byly grafy hezké a velké
plt.rcParams['figure.figsize'] = [12, 6]
%matplotlib inline

PZS Lib načtena úspěšně!


In [6]:
# --- 1. ÚKOL: Tepová frekvence ---

# Načtení záznamu (např. z 'drivedb' - řidiči, stress)
# Příklad záznamu: 'drive01', 'drive02'... pokud nefunguje online, stáhni si je lokálně
record_name = 'drive01' 
db_dir = 'drivedb/1.0.0/'

print(f"Načítám záznam: {record_name}...")
try:
    # Zkusíme stáhnout/načíst
    record = wfdb.rdrecord(record_name, pn_dir=db_dir)
    # Pokud máš data stažená u sebe ve složce data/:
    # record = wfdb.rdrecord(f'data/{record_name}')
except Exception as e:
    print(f"Chyba při načítání: {e}")
    print("Ujisti se, že máš nainstalované wfdb a přístup k internetu, nebo soubor lokálně.")

# Získání dat
fs = record.fs
signal_names = record.sig_name
print(f"Dostupné signály: {signal_names}")
print(f"Vzorkovací frekvence: {fs} Hz")

# Najdeme EKG kanál (často se jmenuje 'ECG' nebo je první)
# V drivedb bývá ECG na indexu 0
ecg_idx = 0 
raw_ecg = record.p_signal[:, ecg_idx]

# Ořízneme to na prvních 60 sekund pro přehlednost (nebo celou hodinu, jak chceš)
limit_sec = 60
limit_samples = limit_sec * fs
sig = raw_ecg[:limit_samples]
t = np.arange(len(sig)) / fs

# Filtrace (odstranění driftu a šumu sítě)
# Lowpass filtr na 40Hz (odstraní šum)
taps = pzs.design_fir_filter(fs, cutoff_freq=40)
sig_filtered = pzs.apply_filter(sig, taps)

# Vizualizace "Před a Po"
pzs.compare_signals(t, sig, sig_filtered, label1="Raw EKG", label2="Filtrované EKG")

Načítám záznam: drive01...


KeyboardInterrupt: 

In [ ]:
# Detekce R-vrcholů
# Heuristika: R-vlna je výrazná špička.
# Práh: 60 % z maxima signálu (můžeš upravit podle kvality signálu)
threshold = 0.6 * np.max(sig_filtered)
# Min. vzdálenost: Mezi tepy musí být aspoň 0.3 sekundy (aby to nebralo T-vlnu jako tep)
min_dist = int(0.3 * fs)

peaks = pzs.find_peaks(sig_filtered, height=threshold, distance=min_dist)

# Výpočet tepovky
diffs_samples = np.diff(peaks)          # Rozdíl v počtu vzorků
diffs_seconds = diffs_samples / fs      # Rozdíl v sekundách
bpm_instant = 60 / diffs_seconds        # Přepočet na BPM

avg_bpm = np.mean(bpm_instant)
print(f"Průměrná tepová frekvence: {avg_bpm:.2f} BPM")

# Graf s vyznačenými vrcholy
plt.figure()
plt.plot(t, sig_filtered, label='EKG')
plt.plot(t[peaks], sig_filtered[peaks], 'rx', markersize=10, label='R-vrcholy')
plt.title(f"Detekce tepu (Průměr: {int(avg_bpm)} BPM)")
plt.xlabel("Čas [s]")
plt.legend()
plt.grid(True)
plt.show()

# Graf vývoje tepu v čase (jak chce zadání)
plt.figure()
# Časová osa pro tepy (bereme čas každého druhého tepu z páru)
t_bpm = t[peaks][1:] 
plt.plot(t_bpm, bpm_instant, 'o-', color='purple')
plt.title("Vývoj tepové frekvence v čase")
plt.xlabel("Čas [s]")
plt.ylabel("Tep [BPM]")
plt.grid(True)
plt.show()